# Dataloader ECL

In [78]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np


Le ECL dataset est un dataset sur la consommation d'electricité de 321 clients toutes les heures entre 2012 et 2014.   
Les colonnes représentent les 321 clients et il y a 26 304 lignes. 
 

In [109]:
ecl_data = pd.read_csv('electricity.csv', header=None).to_numpy()

# Diviser les données en ensembles de train, test et validation
train_data = ecl_data[:18317]
test_data = ecl_data[18317:18317+2633]
val_data = ecl_data[18317+2633:]


## DataLoader

In [110]:
class MonDataLoader(Dataset):
  def __init__(self, data, lookback_size, lookforward_size):
    self.lookback_size = lookback_size
    self.lookforward_size = lookforward_size
    self.data = []
    for i in range(0, 1 + len(data) - (self.lookback_size+ self.lookforward_size), self.lookback_size):
      seq_x = data[i:i+self.lookback_size, :]
      seq_y = data[i+self.lookback_size:i+self.lookback_size+self.lookforward_size, :]
      self.data.append((seq_x, seq_y))

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    seq_x, seq_y = self.data[idx]
    return torch.tensor(seq_x), torch.tensor(seq_y)

#### dataloader

In [111]:
# Diviser les données en ensembles de train, test et validation
train_data = ecl_data[:18317]
test_data = ecl_data[18317:18317+2633]
val_data = ecl_data[18317+2633:]

In [117]:
batch_size = 32

train_dataset = MonDataLoader(ecl_data[:18317], 96, 96)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

eval_dataset = MonDataLoader(ecl_data[18317+2633:], 96, 96)
eval_loader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)

test_dataset = MonDataLoader(ecl_data[18317+2633:], 96, 96)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [121]:
for x,y in test_loader: 
    print(x.shape)
    print(y.shape)


torch.Size([32, 96, 321])
torch.Size([32, 96, 321])
torch.Size([22, 96, 321])
torch.Size([22, 96, 321])
